<hr>
<center>

# Text Mining - Homework 2

</center>
<center>

Authors: 

Timothy, Denis, Pablo

</center>
<hr>


## Part 2 (pipeline example with a small sample of books)

<hr>

Notebook for the creation of the dictionaries with a small subsample of the books dataset, to get the methodology right.

# 0. Packages

In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize  # For tokenizing
from nltk.stem import PorterStemmer  # For stemming
from nltk.stem import LancasterStemmer  # For stemming
from nltk.stem.snowball import SnowballStemmer  # For stemming
from nltk.stem import WordNetLemmatizer  # For lemmatizing
from nltk.corpus import stopwords  # Stopwords list
import re  # For regex expressions
from pandarallel import pandarallel  # For parallelizing pandas row operations
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from typing import Union  # Allows setting as inputs of a function a set of options

# Other utilities
import ast
from collections import Counter
from itertools import chain

# 1. Importing the data

In [ ]:
# # Original (cleaned) data 
# df_og = pd.read_csv('/home/pablo/Downloads/books_and_genres_tim_cleaned.csv')

# # Create a copy to have original dataset in memory, to avoid loading it again
# df = df_og.copy()

In [ ]:
# # Create and save a subsample of books to work with
# df_sample = df_og.sample(200)

# # Save to .csv
# df_sample.to_csv('books_and_genres_tim_cleaned_200sample.csv', index = False)

In [3]:
# Import sample of books
df_sample = pd.read_csv('books_and_genres_tim_cleaned_200sample.csv')

# Drop the language column (all in English)
df_sample.drop(columns = ['lang'], inplace = True)

df_sample.head()

,title,text,genres
0,man to man,Produced by Al Haines\n\n\n\n\n\n\n\n\n\n\n[Fr...,"['adult', 'romance', 'lgbt', 'fiction', 'eroti..."
1,a selection of books,"Produced by Jonathan Ingram, Sankar Viswanatha...","['historical', 'art']"
2,the impostures of scapin,Produced by Delphine Lettau\n\n\n\n\nTHE IMPOS...,"['literature', 'read-for-school', 'school', 'c..."
3,ethics,"Produced by Ted Garvin, David Widger and the D...","['literature', 'classics', 'religion', 'theolo..."
4,speeches literary and social,Transcribed from the 1880 Chatto and Windus ed...,"['classics', 'non-fiction', 'fiction', 'politi..."


In [8]:
print('Before conversion, genre list is stored as a string:\n', type(df_sample.iloc[0, 2]))

Before conversion, genre list is stored as a string:
 <class 'str'>


In [4]:
# Convert the string representation to a list using ast.literal_eval
df_sample['genres'] = df_sample['genres'].apply(ast.literal_eval)

print('After conversion, genre list is stored as a list:\n', type(df_sample.iloc[0, 2]))

After conversion, genre list is stored as a list:
 <class 'list'>


In [5]:
# Flatten all genre lists into one list
all_genres = list(chain.from_iterable(df_sample['genres']))

# Use Counter to get frequencies
genre_counter = Counter(all_genres)

# Convert the counter to a data frame
genre_df = pd.DataFrame(list(genre_counter.items()), columns=['genre', 'frequency'])

# Compute relative frequency of book genres
genre_df['relative_frequency'] = genre_df['frequency'] / len(df_sample)

# Sort the data frame by frequency (highest first)
genre_df = genre_df.sort_values(by='frequency', ascending=False).reset_index(drop = True)

# Display data frame
genre_df.head()

,genre,frequency,relative_frequency
0,fiction,150,0.750
1,classics,109,0.545
2,historical,71,0.355
3,20th-century,65,0.325
4,literature,58,0.290


Note that the sum adds up to more than the number of books in our dataset since a book can be associated to more than one genre.

# 2. Text preprocessing

In [8]:
# Initialize parallelization for pandas
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2.1. Tokenizing and stopword removal

Below, we tokenize the text and lowercase it.

In [9]:
def preprocess_lower(text, rm_stopwords = False, stopword_set = None):
    """
    Preprocess text by:
       - Converting to lowercase.
       - Removing punctuation and digits.
       - Tokenizing.
       - Removing stopwords (optional).
    
    Returns:
        list: A list of tokens lowercased and without punctuation.
    """
    text_lower = text.lower()
    text_no_punct = re.sub(r'[^a-zA-Z\s]', ' ', text_lower)  # Remove digits and punctuation, replace by whitespace (and not by nothing) because of incorrectly spaced punctuation
    tokens = word_tokenize(text_no_punct)
    # Remove stopwords if desired
    if rm_stopwords == True:
        tokens = [token for token in tokens if token not in stopword_set]
    # We return the whole string of tokens so that we can find n-grams later
    return " ".join(tokens)

In [10]:
my_stop_words = set(stopwords.words('english'))

# Create set of custom stopwords (optional)
my_custom_stopwords = {'thou', 'thy', 'thee', 'em', 'er', 'ti', 'la', 'ha',
                       'nay', 'etc', 'hence'}

# Update stopwords (optional)
my_stop_words.update(my_custom_stopwords)

In [11]:
df_sample['text_lower_no_stop'] = df_sample['text'].parallel_apply(
    lambda row: preprocess_lower(text = row, rm_stopwords=True, stopword_set=my_stop_words)
    )

In [43]:
df_sample.head()

,title,text,genres,text_lower_no_stop
0,man to man,Produced by Al Haines\n\n\n\n\n\n\n\n\n\n\n[Fr...,"['adult', 'romance', 'lgbt', 'fiction', 'eroti...",produced al haines frontispiece blazing heat m...
1,a selection of books,"Produced by Jonathan Ingram, Sankar Viswanatha...","['historical', 'art']",produced jonathan ingram sankar viswanathan on...
2,the impostures of scapin,Produced by Delphine Lettau\n\n\n\n\nTHE IMPOS...,"['literature', 'read-for-school', 'school', 'c...",produced delphine lettau impostures scapin les...
3,ethics,"Produced by Ted Garvin, David Widger and the D...","['literature', 'classics', 'religion', 'theolo...",produced ted garvin david widger dp team ethic...
4,speeches literary and social,Transcribed from the 1880 Chatto and Windus ed...,"['classics', 'non-fiction', 'fiction', 'politi...",transcribed chatto windus edition david price ...


## 2.2. Normalization

### A. Stemming

In [44]:
def preprocess_stem(text, stemmer = 'porter'):
    """
    Preprocess text by applying stemming.
    Should just input a string which has been previously pre-processed, which at least removes
    the punctuation.

    Returns:
        str: A string of stemmed tokens separated by spaces.
    """

    tokens = text.split()  # Split input text based on whitespaces

    if stemmer == 'porter':
        stem_class = PorterStemmer()
    elif stemmer == 'lancaster':
        stem_class = LancasterStemmer()
    elif stemmer == 'snowball':
        stem_class = SnowballStemmer("english")
    else:
        print('Stemmer type not accepted. Choose "porter", "lancaster" or "snowball".')

    stemmed_tokens = [stem_class.stem(token) for token in tokens]

    return " ".join(stemmed_tokens)

In [72]:
df_sample['text_stemmed'] = df_sample['text_lower_no_stop'].parallel_apply(
    lambda row: preprocess_stem(text = row, stemmer = 'porter')
    )

In [46]:
# Print sample text
book_index = 10
print('Original text: \n', df_sample['text'][book_index])

Original text: 
 Produced by Greg Weeks, Stephen Blundell and the Online
Distributed Proofreading Team at http://www.pgdp.net









                                           _Illustrated by Paul Orban_

DP

    _Once upon a time life was perfection. Government made sure its
    citizens were supplied with every comfort and pleasure. But
    sometimes perfection breeds boredom and ..._

BY ARTHUR DEKKER SAVAGE


Allen Kinderwood slowed his pace so his forelock would quit bobbing. The
damn thing wasn't supposed to bob; it was supposed to be a sort of
peaked crest above rugged, handsome features--a dark lock brushed
carelessly aside by a man who had more important things to do than fuss
with personal grooming. But no matter how carefully he combed it and
applied lusto-set, it always bobbed if he walked too fast.

But then, why should it matter now? He wasn't looking for a woman
tonight. Not when his appointment with the Social Adjustment counsellors
was tomorrow morning, and he would 

In [47]:
print('Lowercased text, without stopwords: \n', df_sample['text_lower_no_stop'][book_index])

Lowercased text, without stopwords: 
 produced greg weeks stephen blundell online distributed proofreading team http www pgdp net illustrated paul orban dp upon time life perfection government made sure citizens supplied every comfort pleasure sometimes perfection breeds boredom arthur dekker savage allen kinderwood slowed pace forelock would quit bobbing damn thing supposed bob supposed sort peaked crest rugged handsome features dark lock brushed carelessly aside man important things fuss personal grooming matter carefully combed applied lusto set always bobbed walked fast matter looking woman tonight appointment social adjustment counsellors tomorrow morning would get departure permit get one corrected never heard petition dp refused wanted spend last night city main park c sector walking restless crowds trying settle thoughts moved slow aimless eddies brightly appareled citizens avoiding pedestrians skaters heavy four wheeled autoscooters everything dully uncompromisingly sector eve

In [48]:
print('Stemmed text: \n', df_sample['text_stemmed'][book_index])

Stemmed text: 
 produc greg week stephen blundel onlin distribut proofread team http www pgdp net illustr paul orban dp upon time life perfect govern made sure citizen suppli everi comfort pleasur sometim perfect breed boredom arthur dekker savag allen kinderwood slow pace forelock would quit bob damn thing suppos bob suppos sort peak crest rug handsom featur dark lock brush carelessli asid man import thing fuss person groom matter care comb appli lusto set alway bob walk fast matter look woman tonight appoint social adjust counsellor tomorrow morn would get departur permit get one correct never heard petit dp refus want spend last night citi main park c sector walk restless crowd tri settl thought move slow aimless eddi brightli apparel citizen avoid pedestrian skater heavi four wheel autoscoot everyth dulli uncompromisingli sector even size space huge spread tree hope without convict might tini refresh differ anyth mind sap same driven petit illustr allen care brush girl escort sinc 

### B. Lemmatizing

In [12]:
def preprocess_lemmatize(text):
    """
    Preprocess text by applying lemmatizing.
    Should just input a string which has been previously pre-processed, which at least removes
    the punctuation.

    Returns:
        str: A string of lemmatized tokens separated by spaces.
    """

    tokens = text.split()  # Split input text based on whitespaces
    lemmatizer = WordNetLemmatizer()  # Initiallize lemmatizer
    lemmatized_text = []  # Initialize empty list to store lemmatized text
    for word in tokens:
        lemmatized_text.append(lemmatizer.lemmatize(word))

    return " ".join(lemmatized_text)

Lemmatizing with `Spacy` instead of with `WordNetLemmatizer` of `nltk` leads to similar results. In general, however, it seems that there are no significant changes after lemmatizing the text.

In [13]:
df_sample['text_lemmatized'] = df_sample['text_lower_no_stop'].parallel_apply(
    lambda row: preprocess_lemmatize(text = row)
    )

In [56]:
df_sample.head()

,title,text,genres,text_lower_no_stop,text_stemmed,text_lemmatized
0,man to man,Produced by Al Haines\n\n\n\n\n\n\n\n\n\n\n[Fr...,"['adult', 'romance', 'lgbt', 'fiction', 'eroti...",produced al haines frontispiece blazing heat m...,produc al hain frontispiec blaze heat men hors...,produced al haines frontispiece blazing heat m...
1,a selection of books,"Produced by Jonathan Ingram, Sankar Viswanatha...","['historical', 'art']",produced jonathan ingram sankar viswanathan on...,produc jonathan ingram sankar viswanathan onli...,produced jonathan ingram sankar viswanathan on...
2,the impostures of scapin,Produced by Delphine Lettau\n\n\n\n\nTHE IMPOS...,"['literature', 'read-for-school', 'school', 'c...",produced delphine lettau impostures scapin les...,produc delphin lettau impostur scapin le fourb...,produced delphine lettau imposture scapin le f...
3,ethics,"Produced by Ted Garvin, David Widger and the D...","['literature', 'classics', 'religion', 'theolo...",produced ted garvin david widger dp team ethic...,produc ted garvin david widger dp team ethic a...,produced ted garvin david widger dp team ethic...
4,speeches literary and social,Transcribed from the 1880 Chatto and Windus ed...,"['classics', 'non-fiction', 'fiction', 'politi...",transcribed chatto windus edition david price ...,transcrib chatto windu edit david price email ...,transcribed chatto windus edition david price ...


In [52]:
# Print sample text
book_index = 15
print('Original text: \n', df_sample['text'][book_index])

Original text: 
 Produced by Keith G. Richardson




Christ, Christianity and the Bible


BY

I. M. HALDEMAN, D.D.

Pastor First Baptist Church, New York City


_Author of_

How to Study the Bible, The Coming of Christ, The Signs of the
Times, Christian Science in the Light of Holy Scripture, etc., etc.


NEW YORK

CHARLES C. COOK

150 Nassau Street


Copyright, 1912,

By Charles C. Cook


CONTENTS


Christ

Christianity

The Bible



Christ


IF NOT GOD--NOT GOOD


BY I. M. HALDEMAN, D.D.


"Why callest thou me good? there is none good but one, that is God"
(Matthew 9:17).


THE world has accepted Jesus Christ as a good man.

The evidences of his goodness are manifold.

He was full of compassion.

He never looked upon the people as a crowd. He never thought of them
as a mass. He saw them always as individuals. His heart went out to
them. All his impulses were to pity them, sympathize with, and help
them.

He went among them. He entered into all conditions, accepted all
situations. He 

In [53]:
print('Lowercased text, without stopwords: \n', df_sample['text_lower_no_stop'][book_index])

Lowercased text, without stopwords: 
 produced keith g richardson christ christianity bible haldeman pastor first baptist church new york city author study bible coming christ signs times christian science light holy scripture etc etc new york charles c cook nassau street copyright charles c cook contents christ christianity bible christ god good haldeman callest thou good none good one god matthew world accepted jesus christ good man evidences goodness manifold full compassion never looked upon people crowd never thought mass saw always individuals heart went impulses pity sympathize help went among entered conditions accepted situations present wedding ate publicans sinners anon guest rich man table saw ravages disease shame sin tragedies life knew torture body anguish spirit took mystery pain laid upon heart tears meat drink day night became man sorrows expert grief took upon woes world till bowed bent weight years tears sympathy grooved cheeks streams carve way mountain sides men l

In [55]:
print('Lemmatized text: \n', df_sample['text_lemmatized'][book_index])

Lemmatized text: 
 produced keith g richardson christ christianity bible haldeman pastor first baptist church new york city author study bible coming christ sign time christian science light holy scripture etc etc new york charles c cook nassau street copyright charles c cook content christ christianity bible christ god good haldeman callest thou good none good one god matthew world accepted jesus christ good man evidence goodness manifold full compassion never looked upon people crowd never thought mass saw always individual heart went impulse pity sympathize help went among entered condition accepted situation present wedding ate publican sinner anon guest rich man table saw ravage disease shame sin tragedy life knew torture body anguish spirit took mystery pain laid upon heart tear meat drink day night became man sorrow expert grief took upon woe world till bowed bent weight year tear sympathy grooved cheek stream carve way mountain side men looked saw neither form comeliness neithe

## 2.3. Vectorizing - *tf-idf*

In this step, we just vectorize the already-preprocessed text (though we could remove stopwords with the parameters `stop_words`, lowercase the text with `lowercase`, etc.). For more information, check: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

### Example of construction of a simple dictionary (counts of terms)

#### Vectorization

In [107]:
cv = CountVectorizer(
    ngram_range = (1,2),  # Include unigrams, bigrams and trigrams
    min_df=0.05,  # Ignore terms appearing in less than 5% of the documents
    max_df=0.5,  # Ignore terms appearing in more than 50% of the documents 
    lowercase=False, 
    stop_words=None)

# Note that we can fit the count vectorizer with a pandas series
cv.fit(df_sample['text_lower_no_stop'])
vectorized_text = cv.transform(df_sample['text_lower_no_stop'])

# Return dense interpretation of sparse matrix
vectorized_text_dense = vectorized_text.todense()

# Print DTM size
print("Document-term matrix has size", vectorized_text.shape)

# Print terms extracted
terms = cv.get_feature_names_out()
print(terms)

Document-term matrix has size (100, 53634)
['aa' 'aaron' 'ab' ... 'zigzag' 'zone' 'zones']


- **`fit()`** learns the vocabulary from all text in the Series.
- **`transform()`** converts each row into a numerical representation based on that vocabulary.
- The output is a **sparse matrix**, which you convert to dense with `.todense()`.
- **`vectorized_text.shape`** gives the size of the document-term matrix:  
  - Rows = number of documents (i.e., number of books)  
  - Columns = number of unique words in the vocabulary  
- **`cv.get_feature_names_out()`** returns the list of terms that were extracted.

#### Simple dictionary from the counts

In [108]:
# Calculate term frequencies (total counts across all documents)
term_frequencies = vectorized_text.sum(axis=0).A1  # Convert to 1D array

# Create a DataFrame for easier handling
df_terms_description = pd.DataFrame({
    'term': terms,
    'frequency': term_frequencies
})

# Sort the DataFrame by frequency in descending order
df_terms_description = df_terms_description.sort_values(by='frequency', ascending=False).reset_index(drop=True)

# Assign an ID from 1 (most frequent) to V (least frequent)
df_terms_description['id'] = df_terms_description.index + 1

# Display the top 10 terms as a sanity check
df_terms_description.head(10)

,term,frequency,id
0,mrs,3065,1
1,thou,1723,2
2,th,1567,3
3,ye,1448,4
4,thy,1404,5
5,ive,1290,6
6,jo,1237,7
7,roger,1187,8
8,german,1092,9
9,thee,1071,10


### Applying *tf-idf*

In [14]:
def vectorizer(cv: Union[CountVectorizer, TfidfVectorizer], df: pd.DataFrame, column_text: str) -> pd.DataFrame:

    # Note that we can fit the count vectorizer with a pandas series
    cv.fit(df[column_text])
    dtm = cv.transform(df[column_text])  # Create DTM

    # Return dense interpretation of sparse matrix
    dtm_dense = dtm.todense()

    # Print DTM size
    print("Document-term matrix has size", dtm_dense.shape)

    # Save extracted terms
    terms = cv.get_feature_names_out()

    return dtm_dense, terms

According to the notebooks `session4_vectormath` and the one of the 3rd TA session (`vectorization_students_2025`), the way we can replicate the *tf-idf* function seen in class by setting the following parameters: 

- Setting the smoothing parameter to `True` may be useful for preventing zero values whenever there is a term that is included in the matrix but that isn't seen in any document.
- On the other hand, setting the parameter `sublinear_tf=True` is essential to replicate the idea of the regular tf-idf seen in class.

For more information, check https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html. 

In [15]:
# Create a dynamic min_df which is computed as two thirds of the relative frequency 
# of the number of genres for which to construct a dictionary

n_genres = 30  # Number of genres to consider

min_df_top_genres = genre_df['relative_frequency'][n_genres] / 1.5

print(f'Min. df considered for {n_genres} genres: {min_df_top_genres}')

Min. df considered for 30 genres: 0.04


In [76]:
tfidf = TfidfVectorizer(
    lowercase=False,
    stop_words=None,
    sublinear_tf=True,  # Apply tf-idf seen in class 
    smooth_idf=False,
    ngram_range = (1,2),  # Include unigrams, bigrams
    min_df=min_df_top_genres,  # Ignore terms appearing in less than x% of the documents
    max_df=0.5,  # Ignore terms appearing in more than 50% of the documents 
    )

dtm_lower, terms_lower = vectorizer(
    cv = tfidf, df = df_sample, column_text='text_lower_no_stop'
    )

Document-term matrix has size (200, 66683)


In [ ]:
# Step 1: initialize the tfidf vectorizer
tfidf = TfidfVectorizer(
    lowercase=False,
    stop_words=None,
    sublinear_tf=True,  # Apply tf-idf seen in class 
    smooth_idf=False, 
    ngram_range=(1,2),  # Include unigrams, bigrams
    min_df=min_df_top_genres,  # Ignore terms appearing in less than 5% of the documents
    max_df=0.5,  # Ignore terms appearing in more than 50% of the documents 
    )

# Step 2: execute the function with differentt preprocessed descriptions
dtm_stemmed, terms_stemmed = vectorizer(
    cv = tfidf, df = df_sample, column_text='text_stemmed'
    )

Document-term matrix has size (200, 53064)


In [16]:
# Step 1: initialize the tfidf vectorizer
tfidf = TfidfVectorizer(
    lowercase=False,
    stop_words=None,
    sublinear_tf=True,  # Apply tf-idf seen in class 
    smooth_idf=False, 
    ngram_range=(1,2),  # Include unigrams, bigrams
    min_df=min_df_top_genres,  # Ignore terms appearing in less than 5% of the documents
    max_df=0.5,  # Ignore terms appearing in more than 50% of the documents 
    )

# Step 2: execute the function with differentt preprocessed descriptions
dtm_lemmatized, terms_lemmatized = vectorizer(
    cv = tfidf, df = df_sample, column_text='text_lemmatized'
    )

Document-term matrix has size (200, 68951)


In [140]:
dtm_lemmatized

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.01621914],
        [0.        , 0.        , 0.02567191, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00726339,
         0.        ]])

# 3. Dictionary generation

Above, we have created the DTM for all of the books included in the corpus. Now, the idea is to **aggregate the *tf-idf* weights by genre**. Note, however, that this is not straightforward. Some potential issues:
- Adding the weights: *group size effects*. Groups with more documents may naturally have higher summed scores.
- Averaging the weights: averaging the *tf-idf* weights on the whole corpus would again lead to a higher relevance of those genres which are more abundant in the dataset. Therefore, when averaging for the whole corpus we should weight the averages by the relative frequency of a genre in the dataset.

Below, we adopt the last approach, which tends to capture those terms that are more unique for each genre.

In [17]:
# We create a data frame from the dense document-term matrix, with columns named
# the extracted terms
dtm_df = pd.DataFrame(dtm_lemmatized, columns=terms_lemmatized)

# Append the genres column from the original data frame, considering that the
# order of the documents is preserved after applying tf-idf
dtm_df['genres'] = df_sample['genres'].values

dtm_df.head()

,aaron,ab,aback,abandon,abandoning,abandonment,abased,abasement,abashed,abate,...,zero,zest,zeus,zigzag,zinc,zion,zone,zoo,zoological,genres
0,0.0,0.0,0.012258,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.010449,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,"[adult, romance, lgbt, fiction, erotica, conte..."
1,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020709,0.000000,"[historical, art]"
2,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,"[literature, read-for-school, school, classics..."
3,0.0,0.0,0.000000,0.0,0.006497,0.011246,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,"[literature, classics, religion, theology, non..."
4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.009543,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.010557,"[classics, non-fiction, fiction, politics, his..."


Note that the document-term matrix produced by scikit‐learn’s vectorizer preserves the order of the input documents (i.e. the order of rows in the DTM corresponds to the order in the original data frame). This characteristic is considered above to append the genres to the DTM. 

In [18]:
# Keep only the top n genres in a list
top_n_genres = genre_df['genre'][0:n_genres].tolist()
print(top_n_genres)

['fiction', 'classics', 'historical', '20th-century', 'literature', 'non-fiction', 'historical-fiction', 'short-stories', 'novels', 'romance', 'fantasy', 'adult', 'literary-fiction', 'science-fiction', 'adventure', 'childrens', 'biography', 'unfinished', 'young-adult', 'school', 'adult-fiction', 'philosophy', 'american', 'mystery', 'read-for-school', 'religion', 'drama', 'politics', 'contemporary', 'travel']


In [19]:
# Dictionary to store top terms for each genre.
top_terms_by_genre = {}

# Loop over each genre in the top_n_genres list
for genre in top_n_genres:

    # Select rows where the document's genres include the current genre
    genre_mask = dtm_df['genres'].apply(lambda g: genre in g)
    dtm_genre = dtm_df[genre_mask]
    
    # We drop the genres column to work only with numeric tf-idf scores.
    # Then, we aggregate the tf-idf scores for each term across all documents in this genre.
    # Here we use the mean, but for different results we could also use the sum
    # as an aggregation method
    aggregated_scores = dtm_genre.drop(columns=['genres']).mean(axis=0)  # Compute mean across rows
    
    # Sort the aggregated scores in descending order and select the top 30 terms.
    top_30_terms = aggregated_scores.sort_values(ascending=False).head(30)
    
    # Save the result for this genre.
    top_terms_by_genre[genre] = top_30_terms

# Now, top_terms_by_genre is a dictionary where each key is a genre
# and the value is a pandas Series of the top 30 terms (with their aggregated tf-idf scores).
# Display the results:
for genre, series in top_terms_by_genre.items():
    print(f"Top 30 terms for genre: {genre}")
    print(series)
    print("\n")

Top 30 terms for genre: fiction
car           0.008168
said mr       0.007861
maybe         0.007529
honour        0.007502
uncle         0.007474
dollar        0.007414
hotel         0.007126
color         0.007016
colonel       0.006908
grey          0.006739
nodded        0.006697
anyone        0.006694
new york      0.006666
shook head    0.006601
cent          0.006536
aunt          0.006452
minister      0.006403
british       0.006242
funny         0.006223
basket        0.006200
peter         0.006129
james         0.006114
hell          0.006097
anyway        0.006077
christmas     0.006065
worry         0.006058
german        0.006057
candle        0.006035
jumped        0.005985
colour        0.005971
dtype: float64


Top 30 terms for genre: classics
honour       0.009361
grey         0.008213
said mr      0.007935
uncle        0.007636
colour       0.007323
favour       0.006823
holy         0.006802
labour       0.006733
hotel        0.006722
minister     0.006694
ghost   

In [82]:
print(top_terms_by_genre)

{'fiction': illustration    0.010739
em              0.007664
maybe           0.007546
car             0.007404
uncle           0.007339
honour          0.007226
er              0.006979
hotel           0.006870
colonel         0.006838
grey            0.006736
folks           0.006718
dollars         0.006712
nodded          0.006707
anyone          0.006705
new york        0.006671
stairs          0.006622
shook head      0.006498
color           0.006344
said mr         0.006308
funny           0.006250
british         0.006218
aunt            0.006201
sword           0.006158
farm            0.006131
peter           0.006119
james           0.006097
anyway          0.006091
christmas       0.006081
hell            0.006042
jumped          0.006001
dtype: float64, 'classics': honour       0.008984
grey         0.008214
uncle        0.007433
nay          0.007210
em           0.007056
thou         0.007039
devil        0.007002
holy         0.006738
colour       0.006669
stairs      